# P2198

In [1]:
data={"intents":[

    {"tag":"Supplier",
    "patterns":["Damaged HO2S","Damaged powertrain control module (PCM)","Damaged PCM"],
    "responses":["Supplier"],
    "context":[""]
    },
    {"tag":"Dealer",
    "patterns":["Electrical:Short to VPWR in the harness or HO2S","Water in the harness connector","Open/shorted HO2S circuit","Corrosion or incorrect harness connections","Fuel System:Excessive fuel pressure","Leaking/contaminated fuel injectors","Leaking fuel pressure regulator","Low fuel pressure or running out of fuel","Low fuel pressure or running out of fuel","Vapor recovery system","Intake Air System:Air leaks after the mass air flow (MAF) sensor","Vacuum leaks","Positive crankcase ventilation (PCV) system","Improperly seated engine oil dipstick","Leaking gasket","Leaking diaphragm or EGR vacuum regulator","Stuck EGR valve","Oil overfill","Camshaft timing","Cylinder compression"," Exhaust leaks before or near the HO2S(s)"],
    "responses":["Dealer"],
    "context":[""]
    }
]} 

In [2]:
data

{'intents': [{'tag': 'Supplier',
   'patterns': ['Damaged HO2S',
    'Damaged powertrain control module (PCM)',
    'Damaged PCM'],
   'responses': ['Supplier'],
   'context': ['']},
  {'tag': 'Dealer',
   'patterns': ['Electrical:Short to VPWR in the harness or HO2S',
    'Water in the harness connector',
    'Open/shorted HO2S circuit',
    'Corrosion or incorrect harness connections',
    'Fuel System:Excessive fuel pressure',
    'Leaking/contaminated fuel injectors',
    'Leaking fuel pressure regulator',
    'Low fuel pressure or running out of fuel',
    'Low fuel pressure or running out of fuel',
    'Vapor recovery system',
    'Intake Air System:Air leaks after the mass air flow (MAF) sensor',
    'Vacuum leaks',
    'Positive crankcase ventilation (PCV) system',
    'Improperly seated engine oil dipstick',
    'Leaking gasket',
    'Leaking diaphragm or EGR vacuum regulator',
    'Stuck EGR valve',
    'Oil overfill',
    'Camshaft timing',
    'Cylinder compression',
    ' 

In [1]:
import json
data_file = open('P2198.json').read()
intents = json.loads(data_file)

In [2]:
intents

{'intents': [{'tag': 'Supplier',
   'patterns': ['Damaged HO2S',
    'Damaged powertrain control module (PCM)',
    'Damaged PCM'],
   'responses': ['Supplier'],
   'context': ['']},
  {'tag': 'Dealer',
   'patterns': ['Electrical:Short to VPWR in the harness or HO2S',
    'Water in the harness connector',
    'Open/shorted HO2S circuit',
    'Corrosion or incorrect harness connections',
    'Fuel System:Excessive fuel pressure',
    'Leaking/contaminated fuel injectors',
    'Leaking fuel pressure regulator',
    'Low fuel pressure or running out of fuel',
    'Low fuel pressure or running out of fuel',
    'Vapor recovery system',
    'Intake Air System:Air leaks after the mass air flow (MAF) sensor',
    'Vacuum leaks',
    'Positive crankcase ventilation (PCV) system',
    'Improperly seated engine oil dipstick',
    'Leaking gasket',
    'Leaking diaphragm or EGR vacuum regulator',
    'Stuck EGR valve',
    'Oil overfill',
    'Camshaft timing',
    'Cylinder compression',
    ' 

In [3]:
import numpy as np
import random
import json
import torch
import torch.nn as nn
#importing nltk and necessary downloads
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
from torch.utils.data import Dataset, DataLoader
nltk.download('all')

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading all: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [4]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence) 

In [5]:
#using porterstemmer here
stemmer = PorterStemmer()
def stem(word):
    return stemmer.stem(word.lower())

In [6]:
def BagOfWords(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    Bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            Bag[idx] = 1
    return Bag

In [7]:
all_the_words = []
tags = []
pair = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # adding this to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenizing each word in the sentence
        w = tokenize(pattern)
        # addding to our words list(since w is an array we need to use extend for adding the elements)
        all_the_words.extend(w)
        # add to pair
        pair.append((w, tag))

# stem and lower each word. So, first excluding punctuation marks
ignore_words = ['?', '.', '!',',']
all_the_words = [stem(w) for w in all_the_words if w not in ignore_words]
# remove duplicates and sort
all_the_words = sorted(set(all_the_words))
tags = sorted(set(tags))

print(len(pair), "patterns")
print(len(tags), "tags:", tags)
print(len(all_the_words), "unique stemmed words:", all_the_words)
     

24 patterns
2 tags: ['Dealer', 'Supplier']
69 unique stemmed words: ['(', ')', ':', 'after', 'air', 'befor', 'camshaft', 'circuit', 'compress', 'connect', 'connector', 'control', 'corros', 'crankcas', 'cylind', 'damag', 'diaphragm', 'dipstick', 'egr', 'electr', 'engin', 'excess', 'exhaust', 'flow', 'fuel', 'gasket', 'har', 'ho2', 'improperli', 'in', 'incorrect', 'injector', 'intak', 'leak', 'leaking/contamin', 'low', 'maf', 'mass', 'modul', 'near', 'of', 'oil', 'open/short', 'or', 'out', 'overfil', 'pcm', 'pcv', 'posit', 'powertrain', 'pressur', 'recoveri', 'regul', 'run', 's', 'seat', 'sensor', 'short', 'stuck', 'system', 'the', 'time', 'to', 'vacuum', 'valv', 'vapor', 'ventil', 'vpwr', 'water']


In [8]:
# create training data
X_train = []
y_train = []
#using a tuple to run through the pair
for (pattern_sentence, tag) in pair:
    # X-->is the bag of words for each pattern_sentence
    bag = BagOfWords(pattern_sentence, all_the_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot.so defining label
    label = tags.index(tag)
    y_train.append(label)
#array
X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters are--->
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
#len(X_train[0]) means the length of 1st bag of words because they all have the same size. 
#if we want we can just print and check. But its clear here.
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
#lets print the values
print("inputsize=",input_size)
print("outputsize=",output_size)

inputsize= 69
outputsize= 2


In [9]:
#traning dataset
class ChatDataset(Dataset):
#implementing init function
    def __init__(self):
      #storing the values
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [10]:
#creating a new class for our neural network
class NeuralNetModel(nn.Module):
  #This will be a feed forward neural network with two hidden layer 
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetModel, self).__init__()
        #creating the firstlinear layer. This gets the input size and then the connected hiddenlayer
        self.linearlayer1 = nn.Linear(input_size, hidden_size)
        #applying batch normalization
        self.bn1= nn.BatchNorm1d(hidden_size)
        #creating the 1st hidden layer with input size as hiddensize and output size as the hidden size
        self.linearlayer2 = nn.Linear(hidden_size, hidden_size)
        self.bn2= nn.BatchNorm1d(hidden_size) 
        #creating the 2nd hidden layer with input size as hiddensize and output size as the num classes
        self.linearlayer3 = nn.Linear(hidden_size, num_classes)
        #using relu activation function
        self.relu = nn.ReLU()
    #implementing the forward pass
    def forward(self, x):
      #apply our first linear layer which gets x as input and then gives out the output
        output = self.linearlayer1(x)
        #activation function
        output = self.relu(output)
      #apply our first linear layer which gets output as input and then gives out the next output
        output = self.linearlayer2(output)
        #activation function
        output = self.relu(output)
        output = self.linearlayer3(output)
        return output

In [11]:
#suppose if Gpu is available, we can puish our model to the device. otherwise we can have in cpu itself
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNetModel(input_size, hidden_size, output_size).to(device)

In [12]:
#counting the no of parameters it have. 
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)
     

682

In [13]:
#the training dataset, we have defined earlier.
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)
#suppose if Gpu is available, we can puish our model to the device. otherwise we can have in cpu itself
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNetModel(input_size, hidden_size, output_size).to(device)

# Loss is CrossEntropyLoss and optimizer is Adam
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total=0
correct=0
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scores = model(words)
        _, pred = scores.max(1)
        total += len(words)
        correct += (pred==labels).sum()
    if (epoch+1) % 10 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f} And Got {correct} / {total} with accuracy {float(correct)/float(total)*100:.2f}')
print(f'final Accuracy-----> Got {correct} / {total} with accuracy {float(correct)/float(total)*100:.2f}') 
print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_the_words,
"tags": tags
}
#storing these  in a file, this will serialise it and save it to that file named DATA.pth.
FILE = "p2198.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')
     

Epoch [10/1000], Loss: 0.8054 And Got 30 / 240 with accuracy 12.50
Epoch [20/1000], Loss: 0.7225 And Got 64 / 480 with accuracy 13.33
Epoch [30/1000], Loss: 0.5760 And Got 232 / 720 with accuracy 32.22
Epoch [40/1000], Loss: 0.4641 And Got 464 / 960 with accuracy 48.33
Epoch [50/1000], Loss: 0.3320 And Got 694 / 1200 with accuracy 57.83
Epoch [60/1000], Loss: 0.1020 And Got 924 / 1440 with accuracy 64.17
Epoch [70/1000], Loss: 0.2335 And Got 1154 / 1680 with accuracy 68.69
Epoch [80/1000], Loss: 0.0644 And Got 1391 / 1920 with accuracy 72.45
Epoch [90/1000], Loss: 0.1328 And Got 1631 / 2160 with accuracy 75.51
Epoch [100/1000], Loss: 0.0445 And Got 1871 / 2400 with accuracy 77.96
Epoch [110/1000], Loss: 0.0549 And Got 2111 / 2640 with accuracy 79.96
Epoch [120/1000], Loss: 0.0252 And Got 2351 / 2880 with accuracy 81.63
Epoch [130/1000], Loss: 0.0174 And Got 2591 / 3120 with accuracy 83.04
Epoch [140/1000], Loss: 0.0636 And Got 2831 / 3360 with accuracy 84.26
Epoch [150/1000], Loss: 0.0

In [14]:
#loading the data we saved before
FILE = "p2198.pth" 
data = torch.load(FILE)

#defining values
input_size = data["input_size"] 
hidden_size = data["hidden_size"] 
output_size = data["output_size"] 
all_the_words = data['all_words'] 
tags = data['tags'] 
model_state = data["model_state"]

#lets print the model now.
model = NeuralNetModel(input_size, hidden_size, output_size).to(device) 
model.load_state_dict(model_state) 
model.eval()
     

NeuralNetModel(
  (linearlayer1): Linear(in_features=69, out_features=8, bias=True)
  (bn1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linearlayer2): Linear(in_features=8, out_features=8, bias=True)
  (bn2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linearlayer3): Linear(in_features=8, out_features=2, bias=True)
  (relu): ReLU()
)

In [15]:
bot_name = "GM Model"
print("Please enter your cause")
user="P2198"
#name=input("Enter Your defect code ")

while True:
  #once the person types his name, then from the next chat onwards the name will be shown
    sent=input(user+':')
    if sent == "exit":
        break
    else:
        sent = tokenize(sent)
        X = BagOfWords(sent, all_the_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X).to(device)

        output = model(X)
        _, predicted = torch.max(output, dim=1)

        tag = tags[predicted.item()]

        probs = torch.softmax(output, dim=1)
        prob = probs[0][predicted.item()]
        if prob.item() > 0.85:
            for intent in intents['intents']:
                if tag == intent["tag"]:
                    print(f"{bot_name}: {random.choice(intent['responses'])}")
        else:
            print(f"{bot_name}: I do not understand can you please eloborate...")

Please enter your cause
P2198:
GM Model: I do not understand can you please eloborate...
P2198:Electrical
GM Model: Dealer
P2198::
GM Model: Dealer
P2198:HO2S
GM Model: Supplier
P2198:HO2S(s)
GM Model: Supplier
P2198:PCM
GM Model: Supplier
P2198:PCV
GM Model: Dealer
P2198:exit


In [ ]:
GM Model :Dealer 


# P0102

In [16]:
import json
data_file = open('P0102.json').read()
intents = json.loads(data_file)

In [17]:
intents

{'intents': [{'tag': 'OEM',
   'patterns': ['Damaged MAF sensor'],
   'responses': ['OEM'],
   'context': ['']},
  {'tag': 'Dealer',
   'patterns': ['MAF sensor disconnected',
    'MAF circuit open to PCM',
    'VPWR open to MAF sensor',
    'PWR GND open to the MAF sensor',
    'MAF RTN circuit open to PCM',
    'MAF circuit shorted to GND',
    'Intake air leak (near the MAF sensor)'],
   'responses': ['Dealer'],
   'context': ['']}]}

In [18]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence) 

In [19]:
#using porterstemmer here
stemmer = PorterStemmer()
def stem(word):
    return stemmer.stem(word.lower())

In [20]:
def BagOfWords(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    Bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            Bag[idx] = 1
    return Bag

In [3]:
def BagOfords(tokenized_sentence,words):
    sentence_words=[stem(word) for word in tokenized_sentence]
    Bag=np.zeros(len(words),dtype=np.float32)
    for idx,w in enumerate(words):
        if w in sentence_words:
            Bag[idx]=1
        return Bag

SyntaxError: invalid syntax (2341891979.py, line 1)

In [21]:
all_the_words = []
tags = []
pair = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # adding this to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenizing each word in the sentence
        w = tokenize(pattern)
        # addding to our words list(since w is an array we need to use extend for adding the elements)
        all_the_words.extend(w)
        # add to pair
        pair.append((w, tag))

# stem and lower each word. So, first excluding punctuation marks
ignore_words = ['?', '.', '!',',']
all_the_words = [stem(w) for w in all_the_words if w not in ignore_words]
# remove duplicates and sort
all_the_words = sorted(set(all_the_words))
tags = sorted(set(tags))

print(len(pair), "patterns")
print(len(tags), "tags:", tags)
print(len(all_the_words), "unique stemmed words:", all_the_words)
     

8 patterns
2 tags: ['Dealer', 'OEM']
20 unique stemmed words: ['(', ')', 'air', 'circuit', 'damag', 'disconnect', 'gnd', 'intak', 'leak', 'maf', 'near', 'open', 'pcm', 'pwr', 'rtn', 'sensor', 'short', 'the', 'to', 'vpwr']


In [22]:
# create training data
X_train = []
y_train = []
#using a tuple to run through the pair
for (pattern_sentence, tag) in pair:
    # X-->is the bag of words for each pattern_sentence
    bag = BagOfWords(pattern_sentence, all_the_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot.so defining label
    label = tags.index(tag)
    y_train.append(label)
#array
X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters are--->
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
#len(X_train[0]) means the length of 1st bag of words because they all have the same size. 
#if we want we can just print and check. But its clear here.
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
#lets print the values
print("inputsize=",input_size)
print("outputsize=",output_size)

inputsize= 20
outputsize= 2


In [23]:
#traning dataset
class ChatDataset(Dataset):
#implementing init function
    def __init__(self):
      #storing the values
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [24]:
#creating a new class for our neural network
class NeuralNetModel(nn.Module):
  #This will be a feed forward neural network with two hidden layer 
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetModel, self).__init__()
        #creating the firstlinear layer. This gets the input size and then the connected hiddenlayer
        self.linearlayer1 = nn.Linear(input_size, hidden_size)
        #applying batch normalization
        self.bn1= nn.BatchNorm1d(hidden_size)
        #creating the 1st hidden layer with input size as hiddensize and output size as the hidden size
        self.linearlayer2 = nn.Linear(hidden_size, hidden_size)
        self.bn2= nn.BatchNorm1d(hidden_size) 
        #creating the 2nd hidden layer with input size as hiddensize and output size as the num classes
        self.linearlayer3 = nn.Linear(hidden_size, num_classes)
        #using relu activation function
        self.relu = nn.ReLU()
    #implementing the forward pass
    def forward(self, x):
      #apply our first linear layer which gets x as input and then gives out the output
        output = self.linearlayer1(x)
        #activation function
        output = self.relu(output)
      #apply our first linear layer which gets output as input and then gives out the next output
        output = self.linearlayer2(output)
        #activation function
        output = self.relu(output)
        output = self.linearlayer3(output)
        return   output

In [25]:
#suppose if Gpu is available, we can puish our model to the device. otherwise we can have in cpu itself
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNetModel(input_size, hidden_size, output_size).to(device)

In [26]:
#counting the no of parameters it have. 
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)
     

290

In [27]:
#the training dataset, we have defined earlier.
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)
#suppose if Gpu is available, we can puish our model to the device. otherwise we can have in cpu itself
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNetModel(input_size, hidden_size, output_size).to(device)

# Loss is CrossEntropyLoss and optimizer is Adam
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total=0
correct=0
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scores = model(words)
        _, pred = scores.max(1)
        total += len(words)
        correct += (pred==labels).sum()
    if (epoch+1) % 10 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f} And Got {correct} / {total} with accuracy {float(correct)/float(total)*100:.2f}')
print(f'final Accuracy-----> Got {correct} / {total} with accuracy {float(correct)/float(total)*100:.2f}') 
print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_the_words,
"tags": tags
}
#storing these  in a file, this will serialise it and save it to that file named DATA.pth.
FILE = "P0102.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')
     

Epoch [10/1000], Loss: 0.6007 And Got 70 / 80 with accuracy 87.50
Epoch [20/1000], Loss: 0.5641 And Got 140 / 160 with accuracy 87.50
Epoch [30/1000], Loss: 0.5256 And Got 210 / 240 with accuracy 87.50
Epoch [40/1000], Loss: 0.4846 And Got 280 / 320 with accuracy 87.50
Epoch [50/1000], Loss: 0.4400 And Got 350 / 400 with accuracy 87.50
Epoch [60/1000], Loss: 0.3953 And Got 420 / 480 with accuracy 87.50
Epoch [70/1000], Loss: 0.3530 And Got 490 / 560 with accuracy 87.50
Epoch [80/1000], Loss: 0.3145 And Got 560 / 640 with accuracy 87.50
Epoch [90/1000], Loss: 0.2800 And Got 630 / 720 with accuracy 87.50
Epoch [100/1000], Loss: 0.2502 And Got 700 / 800 with accuracy 87.50
Epoch [110/1000], Loss: 0.2243 And Got 770 / 880 with accuracy 87.50
Epoch [120/1000], Loss: 0.2013 And Got 840 / 960 with accuracy 87.50
Epoch [130/1000], Loss: 0.1809 And Got 910 / 1040 with accuracy 87.50
Epoch [140/1000], Loss: 0.1625 And Got 980 / 1120 with accuracy 87.50
Epoch [150/1000], Loss: 0.1496 And Got 1050

In [28]:
#loading the data we saved before
FILE = "P0102.pth" 
data = torch.load(FILE)

#defining values
input_size = data["input_size"] 
hidden_size = data["hidden_size"] 
output_size = data["output_size"] 
all_the_words = data['all_words'] 
tags = data['tags'] 
model_state = data["model_state"]

#lets print the model now.
model = NeuralNetModel(input_size, hidden_size, output_size).to(device) 
model.load_state_dict(model_state) 
model.eval()
     

NeuralNetModel(
  (linearlayer1): Linear(in_features=20, out_features=8, bias=True)
  (bn1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linearlayer2): Linear(in_features=8, out_features=8, bias=True)
  (bn2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linearlayer3): Linear(in_features=8, out_features=2, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "GM Model"
#name=input("Enter Your defect code ")
print("Please enter your cause")
user="P0102"

while True:
  #once the person types his name, then from the next chat onwards the name will be shown
    sent=input(user+':')
    if sent == "exit":
        break
    else:
        sent = tokenize(sent)
        X = BagOfWords(sent, all_the_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X).to(device)

        output = model(X)
        _, predicted = torch.max(output, dim=1)

        tag = tags[predicted.item()]

        probs = torch.softmax(output, dim=1)
        prob = probs[0][predicted.item()]
        if prob.item() > 0.85:
            for intent in intents['intents']:
                if tag == intent["tag"]:
                    print(f"{bot_name}: {random.choice(intent['responses'])}")
        else:
            print(f"{bot_name}: I do not understand the cause can you please eloborate...")

Please enter your cause
